<a href="https://colab.research.google.com/github/ArtificialT800/FoodModel/blob/main/yum_food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [ ]:
import torch
import torchvision
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import os
from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

#Data

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])

train_data = torchvision.datasets.Food101(root="data",
                                          split="train",
                                          download=True,
                                          transform=data_transform)

test_data = torchvision.datasets.Food101(root='data',
                                         split='test',
                                         download=True,
                                         transform=data_transform)

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_dataloader = DataLoader(dataset=train_data, shuffle=True, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(dataset=test_data, shuffle=False, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
len(train_dataloader), len(test_dataloader)

In [ ]:
class_names = train_data.classes

In [ ]:
class CNNModel(nn.Module):
  def __init__(self, input_units, output_units, hidden_units):
    super().__init__()
    self.block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_units, out_channels=hidden_units, kernel_size=1, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=1, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=1, stride=1)
    )

    self.block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=1, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=1, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=1, stride=1)
    )
    self.block_3 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=1, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=1, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=1, stride=1)
    )

    self.Classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*227*227, out_features=output_units)

    )

  def forward(self, x):
    return self.Classifier(self.block_3(self.block_2(self.block_1(x))))

model = CNNModel(input_units=3, hidden_units=10, output_units=len(class_names))
model = model.to(device)

##Loss function and optimizer + helper functions

In [ ]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=0.001)

def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred))*100
  return acc

def time_taken(start: float,
               end: float,
               device: torch.device = device):
  total_time = end-start
  return total_time


In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

start = timer()

epochs = 3

for epoch in tqdm(range(epochs)):
  #Training

  model.train()
  train_loss, train_acc = 0, 0
  for batch, (X, y) in enumerate(train_dataloader):
    X, y = X.to(device), y.to(device)

    y_pred = model(X)

    loss = loss_fn(y_pred, y)
    train_loss += loss

    acc = accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
    train_acc += acc

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  #Testing
  model.eval()
  with torch.inference_mode():
    total_test_loss, total_test_acc = 0, 0
    for batch, (X_test, y_test) in enumerate(test_dataloader):
      X_test, y_test = X_test.to(device), y_test.to(device)

      test_pred = model(X_test)

      test_loss = loss_fn(test_pred, y_test)
      total_test_loss += test_loss

      test_acc = accuracy_fn(y_true=y_test, y_pred=test_pred.argmax(dim=1))
      total_test_acc += test_acc

    total_test_loss /= len(test_dataloader)
    total_test_acc /= len(test_dataloader)

  print(f"Epochs: {epoch}| Train Loss: {train_loss}, Train Accuracy: {train_acc}% | Test Loss: {total_test_loss}, Test Accuracy: {total_test_acc}%")


#Predicting on new/custom images

In [ ]:
def plot_preds(model: torch.nn.Module):
  img_path = input("Enter the image path: ")
  img = torchvision.io.read_image(img_path)
  img = img.type(torch.float32) / 255
  img = img.to(device)
  img_transform = transforms.Compose([
      transforms.Resize(size=(224, 224))
  ])
  img = img_transform(img)
  img = img.unsqueeze(dim=0)

  model = model.to(device)
  model.eval()
  with torch.inference_mode():
    img_pred = model(img)
    img_pred_probs = torch.softmax(img_pred, dim=1)

  image = torch.argmax(img_pred_probs, dim=1).cpu()
  plt.figure(figsize=(12, 7))
  img = img.to("cpu")
  plot_img = img.squeeze()
  plt.imshow(plot_img.permute(1, 2, 0))
  plt.title(f"Pred: {class_names[image]} | Prob: {img_pred_probs.max()}")
  plt.axis(False)
  print(img_pred_probs)

In [ ]:
plot_preds(model=model)

#Save the model if the accuracy is >= 85%

In [ ]:
if int(total_test_acc) >= 85:
  torch.save(obj=model.state_dict(), f='model.pt')
else:
  print("Accuracy isn't good enough...")